In [1]:
import json
import os
import sys
from copy import deepcopy
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import skimage
import tifffile
import yaml

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
from mushroom.mushroom import Mushroom, DEFAULT_CONFIG
import mushroom.data.he as he
import mushroom.data.multiplex as multiplex
import mushroom.utils as utils
import mushroom.visualization.utils as vis_utils

In [5]:
source_root = '/diskmnt/Projects/Users/estorrs/mushroom/data'
target_root = '/data/estorrs/mushroom/data'

In [6]:
run_dir = '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet'

In [7]:
yaml.safe_load(
    open('/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/metadata.yaml')
)

{'resolution': 0.1273782615616727,
 'sections': [{'data': [{'dtype': 'multiplex',
     'filepath': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/HT427PI-A1-U0_multiplex.ome.tiff'}],
   'position': 0,
   'sid': 'HT427PI-A1-U0'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/HT427PI-A1-U1_multiplex.ome.tiff'}],
   'position': 7,
   'sid': 'HT427PI-A1-U1'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/HT427PI-A1-U2_multiplex.ome.tiff'}],
   'position': 15,
   'sid': 'HT427PI-A1-U2'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/diskmnt/Projects/Users/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/HT427PI-A1-U3_multiplex.ome.tiff'}],
   'position': 23,
   'sid': '

## define inputs

In [8]:
fps = sorted(utils.listfiles(run_dir, regex=r'registered/metadata.yaml$'))
fps = [fp for fp in fps if '_20x' in fp]
len(fps), fps

(50,
 ['/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A2_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A3_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A4_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT442PI-A1_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT442PI-A2_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT442PI-A3_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT442PI-A4_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT460P1-A1_20x/registered/metadata.yaml',
  '/data/estorrs/mushroom/data/p

In [24]:
DEFAULT_CONFIG

{'sections': None,
 'dtype_to_chkpt': None,
 'dtype_specific_params': {'visium': {'trainer_kwargs': {'tiling_method': 'radius'}}},
 'sae_kwargs': {'size': 8,
  'patch_size': 1,
  'encoder_dim': 128,
  'codebook_dim': 64,
  'num_clusters': (8, 4, 2),
  'dtype_to_decoder_dims': {'multiplex': (256, 128, 64),
   'he': (256, 128, 10),
   'visium': (256, 512, 2048),
   'xenium': (256, 256, 256),
   'cosmx': (256, 512, 1024),
   'points': (256, 512, 1024)},
  'recon_scaler': 1.0,
  'neigh_scaler': 0.01},
 'trainer_kwargs': {'input_resolution': 1.0,
  'target_resolution': 0.02,
  'pct_expression': 0.05,
  'log_base': 2.718281828459045,
  'tiling_method': 'grid',
  'tiling_radius': 1.0,
  'batch_size': 128,
  'num_workers': 0,
  'devices': 1,
  'accelerator': 'cpu',
  'max_epochs': 1,
  'steps_per_epoch': 1000,
  'lr': 0.0001,
  'out_dir': './outputs',
  'save_every': 1,
  'log_every_n_steps': 10,
  'logger_type': 'tensorboard',
  'logger_project': 'portobello',
  'channel_mapping': {},
  'data

In [25]:
def alter_filesystem(config, source_root, target_root):
    for entry in config['sections']:
        for mapping in entry['data']:
            mapping['filepath'] = mapping['filepath'].replace(source_root, target_root)
    
    if config['trainer_kwargs']['data_mask'] is not None:
        config['trainer_kwargs']['data_mask'] = config['trainer_kwargs']['data_mask'].replace(source_root, target_root)
        
    return config

In [30]:
case_to_config = {}
keep_pcts = [.75, .5, .25, .1]

for pct in keep_pcts:
    for fp in fps:
        metadata = yaml.safe_load(open(fp))
        n_sections = len(metadata['sections'])
        
#         size = int(pct * n_sections)
#         idxs = np.random.choice(np.arange(n_sections), size=size, replace=False)
#         metadata['sections'] = [x for i, x in enumerate(metadata['sections']) if i in idxs]

        case = fp.split('/')[-3]
        name = f'{case}_{pct}keep'

        config = deepcopy(DEFAULT_CONFIG)
        config = utils.recursive_update(config, {
            'sections': metadata['sections'],
            'trainer_kwargs': {
                'input_resolution': metadata['resolution'],
                'target_resolution': .02,
                'out_dir': os.path.join(run_dir, case, 'mushroom', name),
                'accelerator': 'gpu',
                'steps_per_epoch': 1000,
            }
        })

        config['dtype_specific_params'] = utils.recursive_update(config['dtype_specific_params'], {
            'visium': {
                'trainer_kwargs': {
                    'target_resolution': .01  # lower resolution for visium
                }
            }
        })

        config = alter_filesystem(config, source_root, target_root)

        case_to_config[name] = config

In [31]:
case_to_config.keys()

dict_keys(['HT427PI-A1_20x_0.75keep', 'HT427PI-A2_20x_0.75keep', 'HT427PI-A3_20x_0.75keep', 'HT427PI-A4_20x_0.75keep', 'HT442PI-A1_20x_0.75keep', 'HT442PI-A2_20x_0.75keep', 'HT442PI-A3_20x_0.75keep', 'HT442PI-A4_20x_0.75keep', 'HT460P1-A1_20x_0.75keep', 'HT460P1-A2_20x_0.75keep', 'HT460P1-A3_20x_0.75keep', 'HT461B1-A2_20x_0.75keep', 'HT461B1-A3_20x_0.75keep', 'HT462P1-A2_20x_0.75keep', 'HT462P1-A3_20x_0.75keep', 'HT486B1-A1_20x_0.75keep', 'HT491P1-A1_20x_0.75keep', 'HT491P1-A2_20x_0.75keep', 'HT491P1-A4_20x_0.75keep', 'HT495-A2_20x_0.75keep', 'HT495-A3_20x_0.75keep', 'HT495-A4_20x_0.75keep', 'HT497P1-A1_20x_0.75keep', 'HT497P1-A2_20x_0.75keep', 'HT497P1-A3_20x_0.75keep', 'HT502P1-A2_20x_0.75keep', 'HT502P1-A3_20x_0.75keep', 'HT514B1-A2_20x_0.75keep', 'HT514B1-A3_20x_0.75keep', 'HT517B1-A2_20x_0.75keep', 'HT530P1-A1_20x_0.75keep', 'HT530P1-A2_20x_0.75keep', 'HT530P1-A3_20x_0.75keep', 'HT530P1-A4_20x_0.75keep', 'HT535P1-A2_20x_0.75keep', 'HT535P1-A4_20x_0.75keep', 'HT541P1-A3_20x_0.75kee

In [32]:
next(iter(case_to_config.items()))

('HT427PI-A1_20x_0.75keep',
 {'sections': [{'data': [{'dtype': 'multiplex',
      'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/HT427PI-A1-U0_multiplex.ome.tiff'}],
    'position': 0,
    'sid': 'HT427PI-A1-U0'},
   {'data': [{'dtype': 'multiplex',
      'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/HT427PI-A1-U1_multiplex.ome.tiff'}],
    'position': 7,
    'sid': 'HT427PI-A1-U1'},
   {'data': [{'dtype': 'multiplex',
      'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/HT427PI-A1-U2_multiplex.ome.tiff'}],
    'position': 15,
    'sid': 'HT427PI-A1-U2'},
   {'data': [{'dtype': 'multiplex',
      'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT427PI-A1_20x/registered/HT427PI-A1-U3_multiplex.ome.tiff'}],
    'position': 23,
    'sid': 'HT427PI-A1-U3'},
   {'data': [{'dtype': 'multiplex',
      'fil

In [33]:
case_to_config['HT555P1-A3_20x_0.1keep']

{'sections': [{'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT555P1-A3_20x/registered/HT555P1-A3-U15_multiplex.ome.tiff'}],
   'position': 113,
   'sid': 'HT555P1-A3-U15'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT555P1-A3_20x/registered/HT555P1-A3-U24_multiplex.ome.tiff'}],
   'position': 182,
   'sid': 'HT555P1-A3-U24'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT555P1-A3_20x/registered/HT555P1-A3-U31_multiplex.ome.tiff'}],
   'position': 235,
   'sid': 'HT555P1-A3-U31'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/data/projects/submission_v1/lightsheet/HT555P1-A3_20x/registered/HT555P1-A3-U34_multiplex.ome.tiff'}],
   'position': 258,
   'sid': 'HT555P1-A3-U34'},
  {'data': [{'dtype': 'multiplex',
     'filepath': '/data/estorrs/mushroom/

In [38]:
%%time
for case, config in case_to_config.items():
    print(case)
    mushroom = Mushroom.from_config(config)
    mushroom.train()
    mushroom.embed_sections()
    
    z_scaler = .0001
#     for level in range(3):
#         _ = mushroom.generate_interpolated_volumes(z_scaler=z_scaler, level=0, integrated=False)
    _ = mushroom.generate_interpolated_volumes(z_scaler=z_scaler, level=2, integrate=False)
    
    mushroom.save()
    break
    
#     del(mushroom)
    

INFO:root:loading spore for multiplex
INFO:root:data mask detected
INFO:root:starting multiplex processing


HT427PI-A1_20x_0.75keep


INFO:root:using 4 channels
INFO:root:509 sections detected: ['HT427PI-A1-U0', 'HT427PI-A1-U1', 'HT427PI-A1-U2', 'HT427PI-A1-U3', 'HT427PI-A1-U4', 'HT427PI-A1-U5', 'HT427PI-A1-U7', 'HT427PI-A1-U8', 'HT427PI-A1-U9', 'HT427PI-A1-U10', 'HT427PI-A1-U11', 'HT427PI-A1-U13', 'HT427PI-A1-U14', 'HT427PI-A1-U15', 'HT427PI-A1-U16', 'HT427PI-A1-U18', 'HT427PI-A1-U19', 'HT427PI-A1-U21', 'HT427PI-A1-U22', 'HT427PI-A1-U23', 'HT427PI-A1-U26', 'HT427PI-A1-U27', 'HT427PI-A1-U29', 'HT427PI-A1-U32', 'HT427PI-A1-U33', 'HT427PI-A1-U34', 'HT427PI-A1-U35', 'HT427PI-A1-U36', 'HT427PI-A1-U38', 'HT427PI-A1-U39', 'HT427PI-A1-U40', 'HT427PI-A1-U41', 'HT427PI-A1-U42', 'HT427PI-A1-U43', 'HT427PI-A1-U44', 'HT427PI-A1-U45', 'HT427PI-A1-U46', 'HT427PI-A1-U50', 'HT427PI-A1-U51', 'HT427PI-A1-U52', 'HT427PI-A1-U54', 'HT427PI-A1-U55', 'HT427PI-A1-U56', 'HT427PI-A1-U57', 'HT427PI-A1-U58', 'HT427PI-A1-U59', 'HT427PI-A1-U60', 'HT427PI-A1-U61', 'HT427PI-A1-U62', 'HT427PI-A1-U63', 'HT427PI-A1-U64', 'HT427PI-A1-U65', 'HT427PI-A1-

INFO:root:processing sections
INFO:root:generating image data for section HT427PI-A1-U0
INFO:root:generating image data for section HT427PI-A1-U1
INFO:root:generating image data for section HT427PI-A1-U2
INFO:root:generating image data for section HT427PI-A1-U3
INFO:root:generating image data for section HT427PI-A1-U4
INFO:root:generating image data for section HT427PI-A1-U5
INFO:root:generating image data for section HT427PI-A1-U7
INFO:root:generating image data for section HT427PI-A1-U8
INFO:root:generating image data for section HT427PI-A1-U9
INFO:root:generating image data for section HT427PI-A1-U10
INFO:root:generating image data for section HT427PI-A1-U11
INFO:root:generating image data for section HT427PI-A1-U13
INFO:root:generating image data for section HT427PI-A1-U14
INFO:root:generating image data for section HT427PI-A1-U15
INFO:root:generating image data for section HT427PI-A1-U16
INFO:root:generating image data for section HT427PI-A1-U18
INFO:root:generating image data for

INFO:root:generating image data for section HT427PI-A1-U169
INFO:root:generating image data for section HT427PI-A1-U170
INFO:root:generating image data for section HT427PI-A1-U171
INFO:root:generating image data for section HT427PI-A1-U173
INFO:root:generating image data for section HT427PI-A1-U174
INFO:root:generating image data for section HT427PI-A1-U175
INFO:root:generating image data for section HT427PI-A1-U176
INFO:root:generating image data for section HT427PI-A1-U177
INFO:root:generating image data for section HT427PI-A1-U178
INFO:root:generating image data for section HT427PI-A1-U181
INFO:root:generating image data for section HT427PI-A1-U184
INFO:root:generating image data for section HT427PI-A1-U185
INFO:root:generating image data for section HT427PI-A1-U188
INFO:root:generating image data for section HT427PI-A1-U189
INFO:root:generating image data for section HT427PI-A1-U190
INFO:root:generating image data for section HT427PI-A1-U191
INFO:root:generating image data for sect

INFO:root:generating image data for section HT427PI-A1-U363
INFO:root:generating image data for section HT427PI-A1-U365
INFO:root:generating image data for section HT427PI-A1-U366
INFO:root:generating image data for section HT427PI-A1-U369
INFO:root:generating image data for section HT427PI-A1-U370
INFO:root:generating image data for section HT427PI-A1-U371
INFO:root:generating image data for section HT427PI-A1-U372
INFO:root:generating image data for section HT427PI-A1-U373
INFO:root:generating image data for section HT427PI-A1-U374
INFO:root:generating image data for section HT427PI-A1-U375
INFO:root:generating image data for section HT427PI-A1-U376
INFO:root:generating image data for section HT427PI-A1-U377
INFO:root:generating image data for section HT427PI-A1-U378
INFO:root:generating image data for section HT427PI-A1-U379
INFO:root:generating image data for section HT427PI-A1-U380
INFO:root:generating image data for section HT427PI-A1-U383
INFO:root:generating image data for sect

INFO:root:generating image data for section HT427PI-A1-U542
INFO:root:generating image data for section HT427PI-A1-U543
INFO:root:generating image data for section HT427PI-A1-U546
INFO:root:generating image data for section HT427PI-A1-U547
INFO:root:generating image data for section HT427PI-A1-U548
INFO:root:generating image data for section HT427PI-A1-U549
INFO:root:generating image data for section HT427PI-A1-U550
INFO:root:generating image data for section HT427PI-A1-U551
INFO:root:generating image data for section HT427PI-A1-U552
INFO:root:generating image data for section HT427PI-A1-U553
INFO:root:generating image data for section HT427PI-A1-U554
INFO:root:generating image data for section HT427PI-A1-U555
INFO:root:generating image data for section HT427PI-A1-U557
INFO:root:generating image data for section HT427PI-A1-U558
INFO:root:generating image data for section HT427PI-A1-U559
INFO:root:generating image data for section HT427PI-A1-U560
INFO:root:generating image data for sect

Training: |                                                                                         | 0/? [00:…

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

